In [ ]:
import csv
import time
import datetime
import math
import matplotlib.pyplot as plt
from collections import defaultdict

# Preprocess raw annotation data

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

# root = "gdrive/MyDrive/PlushProject/GPT3_explanation_evaluation/annotation/"
root = "../annotation"
explanations_types = ["hateful-WHY", "non-hateful-WHY", "both-WHY" "hateful-COT", "non-hateful-COT", "both-COT" "CONTEXT", ]

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
def takeFirst(elem):
    return int(elem[0])
def takeSecond(elem):
    return int(elem[1])

# Assign labels to each tweet based on a majority voting on three annotations.
def label_based_on_score(hate_score_list):
  newElement = []
  temp = sorted(hate_score_list)

  if(len(temp) >= 2):
    if(temp[-1] >= 4 and temp[-2] >= 4):
      newElement.append("hateful")
      if(len(temp) == 2):
        newElement.append("consistent hateful")
    elif(temp[0] <= 2 and temp[1] <= 2):
      newElement.append("non-hateful")
      if(len(temp) == 2):
        newElement.append("consistent non-hateful")
    else:
      newElement.append("neither")
      if(len(temp) == 2 and temp[0] == 3 and  temp[1] == 3):
        newElement.append("consistent neither")
      elif(len(temp) == 2):
        newElement.append("neither")
      
    if(len(temp) == 3):
      if(temp[-1] >= 4 and temp[-2] >= 4):
        if(temp[0] >= 4):
          newElement.append("consistent hateful")
        else:
          newElement.append("hateful")
      elif(temp[0] <= 2 and temp[1] <= 2):
        if(temp[-1] <= 2):
          newElement.append("consistent non-hateful")
        else:
          newElement.append("non-hateful")
      elif(temp[0] == 3 and temp[1] == 3 and temp[-1] == 3):
          newElement.append("consistent neither")
      else:
        newElement.append("neither")
  else:
    if(temp[0] >= 4):
      newElement.append("hateful")
      newElement.append("consistent hateful")
    elif(temp[0] <= 2):
      newElement.append("non-hateful")
      newElement.append("consistent non-hateful")
    else:
      newElement.append("neither")
      newElement.append("consistent neither")
  return newElement

# Perform preprocessing on the raw files by removing unnecessary columns. In cases where there are two explanations
# , average the two quality scores. Finally, annotate each tweet based on the majority voting from three annotators.
def processData(sourcePath, targetPath, clickWorker):
    title = ["tweetId", "strategyId", "annotationId", "fluency", "informativeness", "persuasiveness", "soundness", \
             "hatefulness", "hateLabel", "consistentHateLabel"]
    leftList = []
    left = 0
    startIndex = 1 if clickWorker else 0

    with open(sourcePath) as fs, open(targetPath, 'w', newline='') as tr:
        writer = csv.writer(tr)
        writer.writerow(title)
        
        reader = csv.reader(fs)
        next(reader)  # Skip header row
        data = list(reader)
        if(clickWorker == True):
          data.sort(key=takeSecond)
        else:
          data.sort(key=takeFirst)

        for i in range(len(data)):
            start = max(i - 2, 0)
            end = min(i + 3, len(data))
            hatredList = []
            for j in range(start, end):
                if(data[j][startIndex + 0] == data[i][startIndex + 0]):
                  hatredList.append(int(data[j][-1]))

            row = data[i]
            newElement = row[0 + startIndex : 3 + startIndex] 
            qualityScore = [int(value) for value in row[-9:-1]]
            if(str(row[startIndex + 1]) == "6" or str(row[startIndex + 1]) == "7"):
              qualityScore = [sum(x) / 2.0 for x in zip(qualityScore[:4] , qualityScore[4:] )] 
            else:
              qualityScore = qualityScore[:4]

            newElement += qualityScore + [row[-1]] + label_based_on_score(hatredList)
            writer.writerow(newElement)

            left += 1
            if((i * 3 + j + 1) % 75 == 0 or (i * 3 + j + 1) == len(data)):
              leftList.append(left)
              left = 0
    return leftList



In [ ]:
explanations_types = ["No_EXP","hateful-WHY", "non-hateful-WHY", "both-WHY", "hateful-COT", "non-hateful-COT", "both-COT", "both-COT_rm_unscroll", "CONTEXT", ]

for exp_typ in explanations_types:
  clickWorker = False
  if(exp_typ == "non-hateful-WHY"):
    clickWorker = True
  processData(root + exp_typ + ".csv", root + exp_typ + "_preprocessed.csv", clickWorker)

# Generate the quality table

In [ ]:

def generateQualityTable(root, target_path):
    explanations_types = ["hateful-WHY", "non-hateful-WHY", "both-WHY", "hateful-COT", "non-hateful-COT", "both-COT", "CONTEXT", ]
    hatelabel = ["non-hateful", "hateful"]
    qualityLabel = ["fluency", "informativeness", "persuasiveness", "soundness" ]
    leftList = [0, 75, 75, 75, 75]
    strategyDic = {}

    title = ["", "Tweets Label"]
    for exp_typ in explanations_types:
      title.append(exp_typ)
      strategyDic[exp_typ] = 0

    with open(target_path, 'w') as file:
      writer = csv.writer(file)
      writer.writerow(title)

      for qi in range(4):
        averageForQuality = 0
        for j in range(2):
          newElement = [qualityLabel[qi], hatelabel[j]]
          for exp_typ in explanations_types:
            with open(root + exp_typ + "_preprocessed.csv") as f:
              reader = csv.reader(f)
              next(reader)
              data = list(reader)
              tweetInterval = data[sum(leftList[:j * 2+1]):  sum(leftList[:j * 2 + 3])]
              qualityList = [float(tweet[3 + qi]) for tweet in tweetInterval]
              qualityScore = sum(qualityList)/ float(len(qualityList))

              strategyDic[exp_typ] += qualityScore
              averageForQuality += qualityScore

              newElement.append(round(qualityScore, 2))
          writer.writerow(newElement) 

generateQualityTable(root, root + "evaluation_table/" + "qualityTable.csv") 

# Generate the hatefulness table

In [ ]:
def generateHatefulnessTable(root, target_path):
    explanations_types = ["No_EXP", "hateful-WHY", "non-hateful-WHY", "both-WHY", "hateful-COT", "non-hateful-COT", "both-COT", "CONTEXT", ]
    hatelabel = ["non-hateful", "hateful"]
    baseline_value = {"non-hateful":0, "hateful":0}
    title = ["hatefulness"]
    for exp_typ in explanations_types:
      title.append(exp_typ)

    with open(target_path, 'w') as file:
      writer = csv.writer(file)
      writer.writerow(title)
      for j in range(2):
        newElement = [hatelabel[j]]
        for exp_typ in explanations_types:
          path = root + exp_typ + "_preprocessed.csv"
                  
          with open(path) as f:
              reader = csv.reader(f)
              next(reader)
              data = list(reader)
              qualityList = [float(tweet[7]) for tweet in data[ j * 150 : (j + 1) * 150]]

              qualityScore = sum(qualityList)/ float(len(qualityList))

              if(exp_typ == "No_EXP"):
                baseline_value[hatelabel[j]] = qualityScore
                newElement.append(round(qualityScore, 2) )
              else:
                difference = qualityScore - baseline_value[hatelabel[j]]
                newElement.append(str(round(qualityScore, 2)) + " (" + str(round(difference, 2)) + ")")

        
        writer.writerow(newElement) 
              
generateHatefulnessTable(root, root + "evaluation_table/" + "hatefulnessTable.csv") 

# Generate the misclassfied table

In [ ]:
hatelabel = ["non-hateful", "hateful", "non-hateful"]

def getMisleadingTable(root, target_path):
    explanations_types = ["hateful-WHY", "non-hateful-WHY", "both-WHY", "hateful-COT", "non-hateful-COT", "both-COT", "both-COT_rm_unscroll" , "CONTEXT", ]
    title = ["", "Transfer Type"]
    for exp_typ in explanations_types:
      title.append(exp_typ)

    with open(target_path, 'w') as file:
      writer = csv.writer(file)
      writer.writerow(title)

      with open(root + "No_EXP_preprocessed.csv") as baseFile:
        reader = csv.reader(baseFile)
        next(reader)
        baseData = list(reader)

        for qi in range(2):
              newElement = ["Misclassified tweets", hatelabel[qi] + " as " + hatelabel[qi+1]]
              for exp_typ in explanations_types:
                path = root + exp_typ + "_preprocessed.csv"
                
                with open(path) as baseFile:
                  reader = csv.reader(baseFile)
                  next(reader)
                  targetData = list(reader)
                  labelList = ["-" for i in range(100)]
                  i = 0
                  while i < len(targetData):
                    end = i+4
                    if(i + 4 >= len(targetData)):
                      end = len(targetData)
                    labelList[int(targetData[i][0]) - 1] = targetData[i][-2]
                    for j in range(i, end):
                      if(int(targetData[j][0]) != int(targetData[i][0])):
                        i = j
                        break
                      if(j == len(targetData) - 1):
                        i = len(targetData)
                        break
                  numOfTweets = int((len(baseData)) / 3.0)

                  total = 0
                  for i in range(numOfTweets):
                    if(baseData[i * 3][-2] == hatelabel[qi] and labelList[i] == hatelabel[qi+1]):
                      total += 1
                  newElement.append(total)

              writer.writerow(newElement) 
getMisleadingTable(root, root + "evaluation_table/" + "misclassifiedTable.csv") 
